# Connect to workspace

In [ ]:
import azureml.core
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()

print(azureml.core.VERSION)

# Create compute cluster

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

compute_name = "cpu-cluster"
compute_min_nodes = 0
compute_max_nodes = 1
vm_size = "STANDARD_DS3_V2"

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('Compute target exists, will reuse it.')
else:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    print(compute_target.get_status().serialize())

# Generate experiment and run it in Azure Machine Learning

Create experiment:

In [ ]:
from azureml.core import Experiment
experiment_name = 'german_credit_data_remote'
experiment = Experiment(ws, experiment_name)

Create Conda environmnent:

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

train_env = Environment("train-env")
train_env.docker.enabled = True
train_env.python.conda_dependencies = CondaDependencies("../config/train-conda.yml")

Load our dataset and create a script run:

In [ ]:
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

dataset = Dataset.get_by_name(ws, name='german_credit_file')

script_run = ScriptRunConfig(source_directory="../", script='train.py')
script_run.run_config.target = compute_target.name
script_run.run_config.environment = train_env
script_run.run_config.arguments = ['--data_path', dataset.as_download(), '--model_name', 'credit-prediction.pkl']

run = experiment.submit(config=script_run)
run

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

In [ ]:
run.get_metrics()

## Register model to workspace

In [ ]:
model = run.register_model(model_name='german-credit-basic-model',
                           model_path='outputs/credit-prediction.pkl',
                           datasets=[['training-dataset', dataset]],
                           tags={'source': 'amlcompute_training_demo'})